**TASK 1**

In [143]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
from sklearn import metrics
import random
import seaborn as sns
from scipy import stats
from copy import deepcopy
from sklearn.cluster import KMeans
from datetime import datetime
import math
import random
import time
from tkinter import *
from statistics import mode 
import statistics 
from statistics import mode
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score

In [ ]:
from google.colab import files
data = files.upload()

Saving football (1).csv to football (1).csv


In [ ]:
data = pd.read_csv('football.csv')
data.head()

,Team,winsInSeason2016,winsInSeason2017
0,X1,3,5
1,X2,3,4
2,X3,2,8
3,X4,2,3
4,X5,6,2


In [ ]:
def loadCSV(fileName):
    fileHandler = open(fileName, "rt")
    lines = fileHandler.readlines()
    fileHandler.close()
    del lines[0] # remove the header
    dataset = []
    for line in lines:
        instance = lineToTuple(line)
        dataset.append(instance)
    return dataset

In [ ]:
def lineToTuple(line):
    # remove leading/trailing witespace and newlines
    cleanLine = line.strip()
    # get rid of quotes
    cleanLine = cleanLine.replace('"', '')
    # separate the fields
    lineList = cleanLine.split(",")
    # convert strings into numbers
    stringsToNumbers(lineList)
    lineTuple = tuple(lineList)
    return lineTuple

In [ ]:
def stringsToNumbers(myList):
    for i in range(len(myList)):
        if (isValidNumberString(myList[i])):
            myList[i] = float(myList[i])

In [ ]:
def isValidNumberString(s):
    if len(s) == 0:
        return False
    if  len(s) > 1 and s[0] == "-":
        s = s[1:]
    for c in s:
        if c not in "0123456789.":
            return False
    return True

In [ ]:
def distance(instance1, instance2):
    if instance1 == None or instance2 == None:
        return float("inf")
    sumOfSquares = 0
    for i in range(0, len(instance1)-1):
        sumOfSquares += (instance1[i+1] - instance2[i])**2
    return sumOfSquares


In [ ]:
def meanInstance(name, instanceList):
    numInstances = len(instanceList)
    if (numInstances == 0):
        return
    numAttributes = len(instanceList[0])
    means = [name] + [0] * (numAttributes)
    for instance in instanceList:
        for i in range(0, numAttributes):
            means[i+1] += instance[i]
   
    for i in range(0, numAttributes):
        means[i+1] /= float(numInstances)
    return tuple(means)

In [ ]:
def assign(instance, centroids, dist):
    minDistance = dist(centroids[0],instance)
    minDistanceIndex = 0
    for i in range(1, len(centroids)):
        d = dist(centroids[i], instance )
        if (d < minDistance):
            minDistance = d
            minDistanceIndex = i
    return minDistanceIndex

In [ ]:
def createEmptyListOfLists(numSubLists):
    myList = []
    for i in range(numSubLists):
        myList.append([])
    return myList

In [ ]:
def assignAll(instances, centroids, labels, dist):
    clusters = createEmptyListOfLists(len(centroids))
    classclusters = createEmptyListOfLists(len(centroids))
    i=0
    for instance in instances:
        clusterIndex = assign(instance, centroids, dist)
        clusters[clusterIndex].append(instance)
        classclusters[clusterIndex].append(labels[i])
        i=i+1
    return clusters, classclusters

In [ ]:
def computeCentroids(clusters):
    centroids = []
    for i in range(len(clusters)):
        name = "CENTROID" + str(i)
        centroid = meanInstance(name, clusters[i])
        centroids.append(centroid)
    return centroids

In [ ]:
def kmeans(instances, k, dist,labels, animation=False, initCentroids=None, oneIteration = False):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = []
    if animation:
        delay = 1.0
        clusters = createEmptyListOfLists(k)
        clusters[0] = instances
    iteration = 0
    while (centroids != prevCentroids):
        if iteration == 1 and oneIteration == True:
          break
        iteration += 1
        clusters, votelabels = assignAll(instances, centroids, labels, dist)
        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = computeWithinss(clusters, centroids, dist)

    print('ITERATIONS: ', iteration)
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["labels"] = votelabels
    return result

In [ ]:
def computeWithinss(clusters, centroids, dist):
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            result += dist(centroid, instance)
    return result


In [ ]:
def repeatedKMeans(instances, k, n):
    bestClustering = {}
    bestClustering["withinss"] = float("inf")
    for i in range(1, n+1):
        print ("k-means trial %d," % i )
        trialClustering = kmeans(instances, k)
        print ("withinss: %.1f" % trialClustering["withinss"])
        if trialClustering["withinss"] < bestClustering["withinss"]:
            bestClustering = trialClustering
            minWithinssTrial = i
    print("Trial with minimum withinss:", minWithinssTrial)
    return bestClustering


In [ ]:
def printTable(instances):
    for instance in instances:
        if instance != None:
            line = instance[0] + "\t"
            for i in range(1, len(instance)):
                line += "%.2f " % instance[i]
            print(line)


In [ ]:
def euclidean(centroid, data):
    sum1=[]
    if len(centroid) == len(data)+1:
        centroid=centroid[1:]
    for i in range (0, len(data)):
        sum1.append((centroid[i]- data[i])**2)
    
    
    euclidean= math.sqrt(sum(sum1))
    return euclidean

In [ ]:
def manhatten(x,y):
    res=[]
    if len(x) == len(y)+1:
        x=x[1:]
    for i in range(len(x)):
        res.append(abs(x[i]-y[i]))
        
    return sum(res) 

In [ ]:
def sse(clustering, centroids):
    i=0
    SSE=0
    for clust in clustering:
        for data in clust:
            SSE=SSE+distance(centroids[i], data)
        i=i+1
    return SSE

In [ ]:
labels =data.Team

In [ ]:
data = data.drop(['Team'], axis=1)

In [ ]:
data

,winsInSeason2016,winsInSeason2017
0,3,5
1,3,4
2,2,8
3,2,3
4,6,2
5,6,4
6,7,3
7,7,4
8,8,5
9,7,6


In [ ]:
a = np.array([[4, 6], [5, 4]])

In [ ]:
dataset = data.values.tolist()

In [ ]:
x = data.winsInSeason2016
y = data.winsInSeason2017

**ANSWER : 1**

In [ ]:
cts = kmeans(dataset, 2, manhatten,labels, True, initCentroids=[[4,6],[5,4]], oneIteration = True)
printTable(cts["centroids"])
print(cts['clusters'])
print('SSE: ',sse(cts['clusters'],cts['centroids']))

ITERATIONS:  1
CENTROID0	4.00 6.33 
CENTROID1	5.57 3.57 
[[[3, 5], [2, 8], [7, 6]], [[3, 4], [2, 3], [6, 2], [6, 4], [7, 3], [7, 4], [8, 5]]]
SSE:  54.09523809523809


**ANSWER : 2**

In [ ]:
cts = kmeans(dataset, 2, euclidean,labels, True, initCentroids=[[4,6],[5,4]], oneIteration = True)
printTable(cts["centroids"])
print(cts['clusters'])
print('SSE: ',sse(cts['clusters'],cts['centroids']))

ITERATIONS:  1
CENTROID0	2.50 6.50 
CENTROID1	5.75 3.88 
[[[3, 5], [2, 8]], [[3, 4], [2, 3], [6, 2], [6, 4], [7, 3], [7, 4], [8, 5], [7, 6]]]
SSE:  47.375


**ANSWER : 3**

In [ ]:
cts = kmeans(dataset, 2, manhatten,labels, True, initCentroids=[[3,3],[8,3]], oneIteration = True)
printTable(cts["centroids"])
print(cts['clusters'])
print('SSE: ',sse(cts['clusters'],cts['centroids']))

ITERATIONS:  1
CENTROID0	2.50 5.00 
CENTROID1	6.83 4.00 
[[[3, 5], [3, 4], [2, 8], [2, 3]], [[6, 2], [6, 4], [7, 3], [7, 4], [8, 5], [7, 6]]]
SSE:  27.833333333333332


**ANSWER 4:**

In [ ]:
cts = kmeans(dataset, 2, manhatten,labels, True, initCentroids=[[3,2],[4,8]], oneIteration = True)
printTable(cts["centroids"])
print(clustering['clusters'])
print('SSE: ',sse(cts['clusters'],cts['centroids']))

ITERATIONS:  1
CENTROID0	4.86 3.57 
CENTROID1	5.67 6.33 
[[[3, 5], [3, 4], [2, 3], [6, 2], [6, 4], [7, 3], [7, 4]], [[2, 8], [8, 5], [7, 6]]]
SSE:  57.904761904761905


**TASK 2**

In [128]:

def kmeans_SSE(inst, k, dist,lab, animation=False, initCentroids=None):
    res = {}
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
        random.seed(time.time())
        centroids = random.sample(inst, k)
    else:
        centroids = initCentroids
    pCentroids = [] 
    newsse= 99999 
    prevsse=1000000 
    if animation:
        delay = 1.0 
        clusters = createEmptyListOfLists(k)
        clusters[0] = inst

    i = 0
    while (new_sse < old_sse):
        i += 1
        clusters, votelabels = assignAll(inst, centroids, lab, dist)
        old_sse = new_sse
        centroids = computeCentroids(clusters)
        new_sse = sse(clusters, centroids)
        withinss = computeWithinss(clusters, centroids, dist)

    print('Number of iterations are ', i)
    print('prev sse', old_sse)
    print('new sse', new_sse)
    res["clusters"] = clusters
    res["centroids"] = centroids
    res["withinss"] = withinss
    res["labels"] = votelabels
    return res

In [164]:
def kmeans_for_hundred(inst, k, dist,lab, animation=False, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        random.seed(time.time())
        centroids = random.sample(inst, k)
    else:
        centroids = initCentroids
    pCentroids = []
    if animation:
        delay = 1.0
        clusters = createEmptyListOfLists(k)
        clusters[0] = inst
      
    i = 0
    while (i<100):
        i +=1
        clusters, votelabels = assignAll(inst, centroids, lab, dist)
       
        pCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = computeWithinss(clusters, centroids, dist)

    print('Number of iterations are ', i)
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["labels"] = votelabels
    return result


In [ ]:
from google.colab import files
data = files.upload()

Saving iris.csv to iris.csv


In [111]:
def cosine(centroid, data):
    if len(centroid) == 5:
        centroid = centroid[1:]
    x1 = centroid
    x1 = np.array(x1)
    x1 = x1.reshape(1,-1)
    y1 = np.array(data)
    y1 = y1.reshape(1,-1)

    ans = cosine_similarity(x1,y1)
    return 1-ans[0][0]

In [117]:
def accuracy(clustering):
    correctpred = 0
    for c in clustering['labels']:
        for label in c:
            if label == mode(c):
                correctpred+=1
    return correctpred/150

In [120]:
def jaccard(centroid, data):
    jac_sum_num=[]
    jac_sum_den = []
    if len(centroid) == 5:
        centroid = centroid[1:]
    for i in range(len(data)):
        jac_sum_num.append(min(centroid[i],data[i]))
        jac_sum_den.append(max(centroid[i],data[i]))
    return 1-(sum(jac_sum_num)/sum(jac_sum_den))

In [96]:
names = [
  'sepal_length',
  'sepal_width',
  'petal_length',
  'petal_width',
  'class',
]

In [98]:
df = pd.read_csv('iris.csv', header=None, names=names) 
df

,sepal_length,sepal_width,petal_length,petal_width,class
0,sepal.length,sepal.width,petal.length,petal.width,variety
1,5.1,3.5,1.4,0.2,Setosa
2,4.9,3,1.4,0.2,Setosa
3,4.7,3.2,1.3,0.2,Setosa
4,4.6,3.1,1.5,0.2,Setosa
...,...,...,...,...,...
146,6.7,3,5.2,2.3,Virginica
147,6.3,2.5,5,1.9,Virginica
148,6.5,3,5.2,2,Virginica
149,6.2,3.4,5.4,2.3,Virginica


In [99]:
labels = df['class'].values.tolist()

In [153]:
df.dropna()
data = df.drop(['class'], axis=1)

In [154]:
dataset = data.values.tolist()
dataset = dataset[1:]

In [155]:
dataset = [[float(x) for x in i] for i in dataset]

In [144]:
cts = kmeans(dataset, 3, cosine,labels, True)
printTable(cts["centroids"])
print('SSE is ',sse(cts['clusters'],cts['centroids']))
print("Accuracy is ",accuracy(cts))

ITERATIONS:  5
CENTROID0	5.01 3.43 1.46 0.25 
CENTROID1	6.54 2.96 5.50 1.99 
CENTROID2	5.94 2.76 4.21 1.30 
SSE is  91.98930917874394
Accuracy is  0.9533333333333334


In [145]:
cts = kmeans(dataset, 3, jaccard,labels, True)
printTable(cts["centroids"])
print('SSE:  ',sse(cts['clusters'],cts['centroids']))
print("Accuracy:  ",accuracy(cts))

ITERATIONS:  7
CENTROID0	5.89 2.74 4.41 1.43 
CENTROID1	5.01 3.43 1.46 0.25 
CENTROID2	6.87 3.09 5.72 2.07 
SSE is  79.09734974533107
Accuracy is  0.8666666666666667


In [156]:
cts = kmeans(dataset, 3, euclidean,labels, True)
printTable(cts["centroids"])
print('SSE: ',sse(cts['clusters'],cts['centroids']))
print("Accuracy: ",accuracy(cts))

ITERATIONS:  5
CENTROID0	5.90 2.75 4.39 1.43 
CENTROID1	5.01 3.43 1.46 0.25 
CENTROID2	6.85 3.07 5.74 2.07 
SSE is  78.85144142614598
Accuracy is  0.8733333333333333


In [157]:
t1 = datetime.now()
cts = kmeans_SSE(dataset, 3, cosine,labels, True)
t2 = datetime.now()
printTable(cts["centroids"])
print("Time Taken: ", t2 - t1)
print('SSE:  ',sse(cts['clusters'],cts['centroids']))
print("Accuracy: ",accuracy(cts))

Number of iterations are  6
prev sse 91.98930917874401
new sse 91.98930917874401
CENTROID0	5.94 2.76 4.21 1.30 
CENTROID1	6.54 2.96 5.50 1.99 
CENTROID2	5.01 3.43 1.46 0.25 
Time Taken:  0:00:00.731479
SSE:   91.98930917874401
Accuracy:  0.9533333333333334


In [158]:
t1 = datetime.now()
cts = kmeans_SSE(dataset, 3, jaccard,labels, True)
t2 = datetime.now()
printTable(cts["centroids"])
print("Time Taken: ", t2 - t1)
print('SSE: ',sse(cts['clusters'],cts['centroids']))
print("Accuracy: ",accuracy(cts))

Number of iterations are  3
prev sse 145.9620085034013
new sse 147.85507308377896
CENTROID0	6.26 2.87 4.91 1.68 
CENTROID1	5.37 3.80 1.52 0.28 
CENTROID2	4.82 3.24 1.43 0.23 
Time Taken:  0:00:00.014644
SSE:  147.85507308377896
Accuracy:  0.66


In [159]:
t1 = datetime.now()
cts = kmeans_SSE(dataset, 3, euclidean,labels, True)
t2 = datetime.now()
printTable(cts["centroids"])
print("Time Taken: ", t2 - t1)
print('SSE: ',sse(cts['clusters'],cts['centroids']))
print("Accuracy: ",accuracy(cts))

Number of iterations are  8
prev sse 78.85144142614602
new sse 78.85144142614602
CENTROID0	6.85 3.07 5.74 2.07 
CENTROID1	5.01 3.43 1.46 0.25 
CENTROID2	5.90 2.75 4.39 1.43 
Time Taken:  0:00:00.021944
SSE:  78.85144142614602
Accuracy:  0.8733333333333333


In [165]:
t1 = datetime.now()
cts = kmeans_for_hundred(dataset, 3, cosine,labels, True)
t2 = datetime.now()
printTable(cts["centroids"])
print("Time Taken: ", t2 - t1)
print('SSE: ',sse(cts['clusters'],cts['centroids']))
print("Accuracy: ",accuracy(cts))

Number of iterations are  100
CENTROID0	5.05 3.65 1.43 0.26 
CENTROID1	6.26 2.87 4.91 1.68 
CENTROID2	4.96 3.21 1.50 0.23 
Time Taken: 0:00:11.632230
SSE:  152.37120000000004
Accuracy:  0.66


In [161]:
t1 = datetime.now()
cts = kmeans_100(dataset, 3, jaccard,labels, True)
t2 = datetime.now()
printTable(cts["centroids"])
print("Time Taken: ", t2 - t1)
print('SSE:',sse(cts['clusters'],cts['centroids']))
print("Accuracy:",accuracy(cts))

Number of iterations are  100
CENTROID0	5.89 2.74 4.41 1.43 
CENTROID1	5.01 3.43 1.46 0.25 
CENTROID2	6.87 3.09 5.72 2.07 
Time Taken:  0:00:00.236571
SSE: 79.09734974533107
Accuracy: 0.8666666666666667


In [166]:
t1 = datetime.now()
cts = kmeans_for_hundred(dataset, 3, euclidean,labels, True)
t2 = datetime.now()
printTable(cts["centroids"])
print("Time Taken: ", t2 - t1)
print('SSE:',sse(cts['clusters'],cts['centroids']))
print("Accuracy: ",accuracy(cts))

Number of iterations are  100
CENTROID0	5.01 3.43 1.46 0.25 
CENTROID1	5.88 2.74 4.39 1.43 
CENTROID2	6.85 3.08 5.72 2.05 
Time Taken:  0:00:00.133000
SSE: 78.85566582597723
Accuracy:  0.8733333333333333
